In [1]:
import nltk
import random
import os
from sklearn import svm
import json
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score

In [2]:
label_map={'neg':0 ,'sli_neg':1 ,'neutral':2 ,'sli_pos':3 ,'pos':4}

In [3]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,label_map[each_file]) for sentence in data_temp]
    return dataset

In [4]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
with open('../NaiveBayes/vocab_bigram') as f:
    vocab=json.load(f)    

In [5]:
#bigrams
print labeled_data[:5]

[(u'also learn balanc life', 2), (u'nice work', 4), (u'tight deadlin', 1), (u'time simpli moder answer question', 2), (u'manag care number well', 3)]


In [6]:
print len(labeled_data)

201378


In [8]:
print len(vocab)
print vocab[:10]

10863
[u'team set', u'still get', u'work abl', u'logist team', u'good reput', u'year employ', u'time realli', u'quickli learn', u'custom experi', u'steep learn']


In [9]:
vectorizer = CountVectorizer(vocabulary=vocab,min_df=1,ngram_range=(2,2))
corpus=[sentence for sentence,pol in labeled_data]
features=vectorizer.fit_transform(corpus[:80000])
labels=[label for sentence,label in labeled_data[:80000]]

In [10]:
classifier=svm.SVC(kernel='linear',probability=True)

In [14]:
classifier.fit(features[:50000],labels[:50000])
#with open('svc_bigram_model','w') as f:
#    pickle.dump(classifier,f)

In [15]:
#accuracy
predicted = classifier.predict(features[50000:65000])
accuracy=accuracy_score(labels[50000:65000],predicted)
#with open('acuracy_svc_bigram','w') as f:
#    json.dump(accuracy,f) 

In [16]:
with open('acuracy_svc_bigram') as f:
    accuracy=json.load(f) 
print accuracy

0.699333333333


In [35]:
#testing
with open('svc_bigram_model') as f:
    classifier=pickle.load(f)
feature_test=[]
print 'sample :'+corpus[51002]
# creating feature set
for word in vocab: 
    if word in corpus[51002]:
        feature_test.append(1)
    else:
        feature_test.append(0)
#predicting class        
classifier.predict(feature_test)

sample :realli enjoy work


/home/pranav/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([3])